In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
from torch.utils.data import Dataset, DataLoader
import wandb
import torch.nn as nn
import torch.optim as optim

# Loading, scaling the data
data has been explored and concaternated in `data_preparerationipynb`

In [4]:
# Load the dataset
df = pd.read_csv('../data/train.csv')
df.shape

(2000, 21)

In [5]:

# Separate the target variable from the features
y = df['price_range'].values
X = df.drop('price_range', axis=1)

scaler = StandardScaler()
X = scaler.fit_transform(X)
# y = scaler.fit_transform(y.values.reshape(-1, 1))

#  Split the data into train, validation, and test sets 0.7, 0.15, 0.15
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

# # Create scaling pipeline for the features
# X_train = scaler.fit_transform(X_train)
# X_val = scaler.transform(X_val)
# X_test = scaler.transform(X_test)

print(X_train.shape, X_val.shape, X_test.shape)
print(y_train.shape, y_val.shape, y_test.shape)


(1400, 20) (300, 20) (300, 20)
(1400,) (300,) (300,)


# Dataset

In [6]:
class PhoneDataset(Dataset):
    def __init__(self, X, y):
        # Convert inputs to torch 32 bit float tensor
        X = torch.tensor(X, dtype=torch.float32)
        y = torch.tensor(y, dtype=torch.float32)

        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        if idx < 0 or idx >= len(self):
            raise IndexError(f"Index {idx} is out of range")

        return self.X[idx], self.y[idx]

In [7]:
# train_dataset = PhoneDataset(X_train, y_train)
# val_dataset = PhoneDataset(X_val, y_val)
#
# train_dataset[2]

Forward feed neural network


In [8]:
class FFNN(nn.Module):
    def __init__(self):
        super(FFNN, self).__init__()
        self.fc1 = nn.Linear(in_features=20, out_features=10)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(in_features=10, out_features=5)
        self.fc3 = nn.Linear(in_features=5, out_features=1)

        self.softmax = torch.nn.Softmax(dim=1)

    def init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                nn.init.zeros_(m.bias)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)



        x = self.softmax(x)
        return x

# Loss function

In [9]:
# how to evaluate model with wandb
def train(args, model, device, train_loader, optimizer, criterion, epoch):
    # Switch model to training mode. This is necessary for layers like dropout, batchnorm etc which behave differently in training and evaluation mode
    model.train()

    # We loop over the data iterator, and feed the inputs to the network and adjust the weights.
    for inputs, targets in train_loader:
        # Zero the gradients
        optimizer.zero_grad()

        # Feed the inputs to the network
        outputs = model(inputs)

        # Compute the loss
        loss = criterion(outputs, targets)

        # Backpropagate the gradients
        loss.backward()

        # Update the weights
        optimizer.step()


In [24]:
def test(args,   model, device, criterion, test_loader):
    # Switch model to evaluation mode. This is necessary for layers like dropout, batchnorm etc which behave differently in training and evaluation mode
    model.eval()
    test_loss = 0
    correct = 0

    example_images = []
    with torch.no_grad():
        for data, target in test_loader:
            # Load the input features and labels from the test dataset
            data, target = data.to(device), target.to(device)

            # Make predictions: Pass image data from test dataset, make predictions about class image belongs to (0-9 in this case)
            output = model(data)

            # Compute the loss sum up batch loss
            test_loss += criterion(output, target).item()

            # Get the index of the max log-probability
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
            # print(pred)


    # WandB – wandb.log(a_dict) logs the keys and values of the dictionary passed in and associates the values with a step.
    # You can log anything by passing it to wandb.log, including histograms, custom matplotlib objects, images, video, text, tables, html, pointclouds and other 3D objects.
    # Here we use it to log test accuracy, loss and some test images (along with their true and predicted labels).
    data = {"Test Accuracy": 100. * correct / len(test_loader.dataset), "Test Loss": test_loss}
    wandb.log(data)

# Config

In [16]:
config = wandb.config  # Initialize config

config.batch_size = 4          # input batch size for training (default: 64)
config.test_batch_size = 10    # input batch size for testing (default: 1000)
config.epochs = 50             # number of epochs to train (default: 10)
config.lr = 0.1               # learning rate (default: 0.01)
config.momentum = 0.1          # SGD momentum (default: 0.5)
# config.no_cuda = True         # disables CUDA training
config.seed = 42               # random seed (default: 42)
config.log_interval = 10

# Data loader

In [ ]:
# Create the data loaders
train_dataset = PhoneDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=config.batch_size, shuffle=True)

val_dataset = PhoneDataset(X_val, y_val)
val_loader = DataLoader(val_dataset, batch_size=config.test_batch_size, shuffle=True)

test_dataset = PhoneDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, shuffle=True)

In [18]:
# use_cuda = not config.no_cuda and torch.cuda.is_available()
# device = torch.device("cuda" if use_cuda else "cpu")
device  = torch.device("cpu")
kwargs = {}

torch.manual_seed(config.seed)

model = FFNN().to(device)

loss = nn.MSELoss()

optimizer = optim.SGD(model.parameters(), lr=config.lr, momentum=config.momentum)

wandb.watch(model, log="all")

[]

In [2]:
wandb.init(project="phone-price-prediction")

for epoch in range(1, config.epochs + 1):
    train(config, model, device, train_loader, optimizer, loss, epoch)
    test(config, model, device, loss , val_loader)

wandb.finish()


NameError: name 'wandb' is not defined

Test Accuracy,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Test Loss,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Test Accuracy,23.66667
Test Loss,48.6
